# **Build Instance Graph (BIG) Algorithm**

The code has been developed with the pm4py library, a set of functions used for process mining tasks

In [ ]:
!pip install -U pm4py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Find Causal Relationship
This function finds the casual relationships from the petri-net of the process model.
The inputs are the petri-net, the initial marking, and the final marking.
The output is a list of python pairs where the second element directly follows the first element in the petri-net, which translates to a causal relationship.

In [ ]:
from pm4py.algo.discovery.footprints import algorithm as footprints_discovery
def findCausalRelationships(net, im, fm):
  fp_net = footprints_discovery.apply(net, im, fm)
  return list(fp_net.get('sequence'))

##Extract Instance Graph

This function extracts the instance graph of a trace. Every event of the trace is saved in a list V which represents the set of the nodes of the graph. An event is a pair of an ID (generated incrementally) and the activity label. The edges instead are saved as a pair of events in a list W. The algorithm is based on the definition 18 of the original paper.

In [ ]:
def ExtractInstanceGraph(trace, cr):
    V = []
    W = []
    id = 1
    for event in trace:
      V.append((id, event.get("concept:name")))
      id += 1
    # print("IG")
    for i in range(len(V)):
      for k in range(i+1,len(V)):
        e1 = V[i]
        e2 = V[k]
        '''if e1[0]==e2[0]:
          continue;'''
        if (e1[1],e2[1]) in cr:
          flag_e1=True
          for s in range(i+1, k):
            e3 = V[s]
            if (e1[1],e3[1]) in cr:
              flag_e1 = False
              break
          flag_e2=True
          for s in range(i+1, k):
            e3 = V[s]
            if (e3[1],e2[1]) in cr:
              flag_e2 = False
              break
          
          if flag_e1 or flag_e2:
            W.append((e1,e2))
    return V, W

##Check Trace Conformance

This function checks whether a given trace is conform to a petri-net through alignment based conformance checking. It takes in input the trace, the petri-net, the initial marking, and the final marking.
It returns two lists: a list of sequences of deleted activities, and a list of sequences of inserted activities in the trace. 
Each element of the two lists is a list itself, this sublist contains pairs where the first element is the position of the deleted/inserted activity and the second is the activity label. In sequences of deleted activities the position value is always the same, while for inserted activities it's incremental.

In [ ]:
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments


def checkTraceConformance(trace, net, initial_marking, final_marking):
  
  aligned_traces = alignments.apply_trace(trace, net, initial_marking, final_marking)
  D = []
  I = []
  id = 0
  temp_d = []
  temp_i = []
  prev_d = False
  curr_d = False
  prev_i = False
  curr_i = False
  del_count = 1
  for edge in aligned_traces['alignment']:
    id+=1
    if edge[1] is None:
      id-=1
      continue
    if edge[0] == '>>':
      temp_d.append((id, edge[1]))
      curr_d = True
      id-=1
    if edge[1] == '>>':
      temp_i.append((id, edge[0]))
      curr_i = True
    
    if (prev_i and not curr_i):
      if len(temp_i) > 0:
        I.append(temp_i)
      temp_i = []
    prev_i = curr_i
    curr_i = False
    if (prev_d and not curr_d):
      if len(temp_d) > 0:
        D.append(temp_d)
      temp_d = []
      
    prev_d = curr_d
    curr_d = False
  if len(temp_i) > 0:
      I.append(temp_i)
  if len(temp_d) > 0:
      D.append(temp_d)
  return D, I   


##View Instance Graph

This function takes in input the list of nodes (events) and the list of edges and returns a GraphViz object. The other two inputs are view and the title of the graph. By default view is true so the function will show the graph in the output window.


In [ ]:
from IPython import display
from graphviz import Digraph
def viewInstanceGraph(V, W, view=True, title="Instance Graph"):
  # Conversion to string indexes
  V2 = []
  W2 = []
  for node in V:
    V2.append((str(node[0]), "{0} = {1}".format(node[0],node[1])))
  for edge in W:
    W2.append(((str(edge[0][0]), "{0} = {1}".format(edge[0][0],edge[0][1])),(str(edge[1][0]), "{0} = {1}".format(edge[1][0],edge[1][1]))))

  dot = Digraph(comment=title, node_attr={'shape': 'circle'})
  for e in V2:
    dot.node(e[0], e[1])
  for w in W2:
    dot.edge(w[0][0], w[1][0])
  if view:
    display.display(dot)
  return dot


##Irregular Graph Repairing

This is the central function of the BIG Algorithm. Given an irregular graph and the lists of deleted (D) and inserted (I) activities, the algorithm returns a repaired graph. This function repairs first all the deleted activities, and then all the inserted activities, calling the DeletionRepair function and the InsertionRepair function respectively.

In [ ]:
def irregularGraphRepairing(V, W, D, I, cr, view=False):
  Wi=W
  all_deleted_labels = []
  for d_element in D:
      for element in d_element:
        if element[1] not in all_deleted_labels:
          all_deleted_labels.append(element[1])
  for d_element in D:
    Wi=DeletionRepair(Wi, V, d_element,cr, all_deleted_labels)
  if view:
    print("Deletion repaired Instance Graph")
  graph = viewInstanceGraph(V, Wi, view)

  all_inserted = []
  for i_element in I:
    for i in i_element:
      if i not in all_inserted:
        all_inserted.append(i)   
  for i_elements in I:
    Wi=InsertionRepair(Wi,V,i_elements,cr, all_inserted)
  if view:
    print("Insertion repaired Instance Graph")
  graph = viewInstanceGraph(V, Wi, view) 
  return Wi, graph

###Is Reachable

This is a boolean function used in both DelationRepair and InsertionRepair functions. It takes as input the instance graph and two events (source and destination) and checks if a path from the source and the destination events exists in the graph.

In [ ]:
def isReachable(V, W, s, d):
  # Mark all the vertices as not visited
  visited =[False]*(len(V))
  
  # Create a queue for BFS
  queue=[]
  
  # Mark the source node as visited and enqueue it
  queue.append(s)
  visited[s[0] -1] = True

  while queue:
    
    #Dequeue a vertex from queue
    j = queue.pop(0)
    
    # If this adjacent node is the destination node, then return true
    if j == d:
      return True
 
    # Else, continue to do BFS
    for edge in W:
      if edge[0] == j:
        if visited[edge[1][0] - 1] == False:
          queue.append(edge[1])
          visited[edge[1][0] - 1] = True
  
  # If BFS is complete without visited d
  return False


###Deletion Repair

The idea underlying this function consists in connecting activities that occurred before and after the deleted activities, and in removing those edges which should not have been created according to the petri-net. For more information, see sub-chapter 5.1 of the reference paper.

In [ ]:
def DeletionRepair(Wi, V, d_elements, cr, all_deleted):
  v_len = len(V)
  Wr1 = []
  Wr2 = []
  i = d_elements[0][0]

  if i <= v_len:  
    for edge in Wi:
      if edge[1][0] == i and edge[0][0] < i and (d_elements[-1][1],V[i-1][1]) in cr:
        for h in range(edge[0][0], i):
          if (V[h-1][1],d_elements[0][1]) in cr:
            Wr1.append(edge)
            break

      if edge[0][0] < i and edge[1][0] > i and (d_elements[-1][1],edge[1][1]) in cr:
        if edge[0][1] in all_deleted:
          Wr2.append(edge)
        elif (edge[0][1],d_elements[0][1])  in cr:
          for l in range(i+1, edge[1][0]):
            if (V[l-1],edge[1]) in Wi:
              Wr2.append(edge)
              break

  Wi = list(set(Wi) - set(Wr1 + Wr2))
  for k in range(i - 1, 0, -1):
    for j in range(i, v_len+1):
      if (V[k-1][1],d_elements[0][1]) in cr:
        if (d_elements[-1][1], V[j-1][1]) in cr:
          if not isReachable(V, Wi, V[k-1], V[j-1]):
            flag1 = True
            for l in range(k + 1, j):
              if (V[k-1],V[l-1]) in Wi:
                flag1 = False
                break
            flag2 = True
            for m in range(k + 1, i):
              if (V[m-1],V[j-1]) in Wi:
                flag2 = False
                break
            if flag1 or flag2:
              Wi.append((V[k-1],V[j-1]))
  return Wi


###Insertion Repair

This function is aimed at restructuring an irregular graph when a sequence of inserted activities is detected. For more information, see sub-chapter 5.2 of the original paper.

In [ ]:
def InsertionRepair(W, V, i_elements, cr, all_inserted):
  v_len = len(V)
  Wr1=[]
  Wr2=[]
  Wr3=[]
  Wr4=[]
  Wr5=[]
  Wa1=[]
  Wa2=[]
  Wa3=[]
  i= i_elements[0][0]
  j=i+len(i_elements)-1
  Wi=W.copy()
  
  for edge in Wi:
    if edge[0][0]<i and edge[1][0]>=i and edge[1][0]<=j:
      Wr1.append(edge)
    if edge[0][0]>=i and edge[0][0]<=j and edge[1][0]>j:
      Wr2.append(edge)
    if edge[0][0]>=i and edge[0][0]<=j and edge[1][0]>=i and edge[1][0]<=j:
      Wr3.append(edge)
  Wi= list(set(Wi) - set(Wr1 + Wr2 + Wr3))

  for k in range(j+1, v_len+1):     
      if V[k-1] not in all_inserted:
        if (V[i-2][1],V[k-1][1]) in cr or (V[i-2],V[k-1]) in Wi:
          if not isReachable(V, Wi, V[j-1], V[k-1]):
            Wi.append((V[j-1],V[k-1]))
            Wa1.append((V[j-1],V[k-1]))

  # if i < v_len and (V[i-2][1],V[i][1]) not in cr:
  if i == v_len or (V[i-2][1],V[i][1]) not in cr:
      Wi.append((V[i-2],V[i-1]))
      Wa2.append((V[i-2],V[i-1]))
  else:
    for k in range(i-1,0,-1):
        if V[k-1] not in all_inserted:
          if j < v_len and ((V[k-1][1],V[j][1]) in cr or (V[k-1],V[j]) in Wi):
            if not isReachable(V, Wi, V[k-1],V[i-1]):
              Wi.append((V[k-1],V[i-1]))
              Wa2.append((V[k-1],V[i-1]))
                    
  for k in range(i, j):
    Wa3.append((V[k-1],V[k]))
  if len(Wa3)>0:
      Wi=Wi+Wa3

  for edge in Wa2:
    for edge2 in Wa1:
      if edge[1][0]>=i and edge[1][0]<=j:
        if edge2[0][0]>=i and edge2[0][0]<=j:
          Wr4.append((edge[0],edge2[1]))
  Wi= list(set(Wi) - set(Wr4)) 
  
  # if i < v_len and (V[i-2][1],V[i][1]) not in cr
  if i == v_len or (V[i-2][1],V[i][1]) not in cr:
    for edge in Wi:   
      if edge[1][0]>i and edge[0][0]==i-1:
        Wr5.append(edge)
    Wi = list(set(Wi) - set(Wr5))
  return Wi

##Save GFile

The SaveGFile function saves the repaired graph generated from the BIG algorithm in the .g format with extra information relatively the execution time of the algorithm and the eventual repaired events (deleted or inserted).
The saveGfinal function instead saves all the repaired graph generated in a single .g format file.

In [ ]:
def saveGFile(V, W, path, D, I, time, sort_labels):
  with open(path, 'w') as f:
    f.write("# Execution Time: {0:.3f} s\n".format(time))
    f.write("# Deleted Activities: {0}\n".format(D))
    f.write("# Inserted Activities: {0}\n".format(I))
    for n in V:
      f.write("v {0} {1}\n".format(n[0], n[1]))
    f.write("\n")
    if (sort_labels):
      W.sort()
    for e in W:
      f.write("e {0} {1} {2}__{3}\n".format(e[0][0], e[1][0], e[0][1], e[1][1]))


In [ ]:
def saveGfinal(V, W, path, sort_labels):
  with open(path, 'a') as f:
    f.write("XP \n")
    for n in V:
      f.write("v {0} {1}\n".format(n[0], n[1]))
    if (sort_labels):
      W.sort()
    for e in W:
      f.write("e {0} {1} {2}__{3}\n".format(e[0][0], e[1][0], e[0][1], e[1][1]))
    f.write("\n")
    f.close()
    


##Main function

This block puts together the previous functions to form the complete BIG algorithm.
It takes in input, the path of the petri-net, the path of the xes log, the starting trace (not the trace id but the position in the log), the ending trace, view (to enable to visualize the instance graphs in the output window), and sort_labels(to sort the labels of the edges in the .g files). It will also save the graphviz and the .g files for each correct/repaired graph and one for all the instance graph.


In [ ]:
from pm4py.streaming.importer.xes import importer as xes_importer
import time
from pm4py.objects.petri_net.importer import importer as pnml_importer

import graphviz

from pm4py.visualization.petri_net import visualizer as pn_visualizer


log_file = "/content/drive/MyDrive/DWH/BIG Datasets/BPI2017Denied/BPI2017Denied.xes"
net_file = "/content/drive/MyDrive/DWH/BIG Datasets/BPI2017Denied/BPI2017Denied_petriNet.pnml"
log_file2 = "/content/drive/MyDrive/DWH/BIG Datasets/testBank2000NoRandomNoise/testBank2000NoRandomNoise.xes"
net_file2 = "/content/drive/MyDrive/DWH/BIG Datasets/testBank2000NoRandomNoise/testBank2000NoRandomNoise_petriNet.pnml"
log_file3 = "/content/drive/MyDrive/DWH/BIG Datasets/Hospital_dcc/Hospital_dcc.xes"
net_file3 = "/content/drive/MyDrive/DWH/BIG Datasets/Hospital_dcc/Hospital_dcc_petriNet.pnml"
log_file4 = "/content/drive/MyDrive/DWH/BIG Datasets/bpi2012decompositionExpr/bpi2012decompositionExpr.xes"
net_file4 = "/content/drive/MyDrive/DWH/BIG Datasets/bpi2012decompositionExpr/bpi2012decompositionExpr_petriNet.pnml"
log_file5 = "/content/drive/MyDrive/DWH/BIG Datasets/testBank2000SCCUpdatedCopia/testBank2000SCCUpdatedCopia.xes"
net_file5 = "/content/drive/MyDrive/DWH/BIG Datasets/testBank2000SCCUpdatedCopia/testBank2000SCCUpdatedCopia_petriNet.pnml"
log_file6 = "/content/drive/MyDrive/DWH/BIG Datasets/toyex.xes"
net_file6 = "/content/drive/MyDrive/DWH/BIG Datasets/toyex_petriNet.pnml"

def BIG(net_path, log_path, tr_start=0, tr_end=None, view=False, sort_labels=False):

  splits = log_path.split('/')
  name = splits[-1].split(".")[0]

  streaming_ev_object = xes_importer.apply(log_path, variant=xes_importer.Variants.XES_TRACE_STREAM)
  net, initial_marking, final_marking = pnml_importer.apply(net_path)

  gviz = pn_visualizer.apply(net, initial_marking, final_marking)
  display.display(gviz)
  gviz.render(filename="petri")

  start_time_total = time.time()
  cr = findCausalRelationships(net, initial_marking, final_marking)
  if view:
    print(cr)

  count = 0
  repairs = 0
  for trace in streaming_ev_object:
    count += 1
    if count < tr_start:
      continue
    elif tr_end is not None and count > tr_end:
      break

    num = trace.attributes.get('concept:name')
    trace_start_time = time.time()
    V, W = ExtractInstanceGraph(trace,cr)
    if view:
      print("\n\n------------------------------------\nUnrepaired Instance Graph")
      print(V)
    graph = viewInstanceGraph(V, W, view)
    D, I = checkTraceConformance(trace,net,initial_marking, final_marking)
    print("Count {0}, Len {1}, Trace name: {2}".format(count, len(V), num))
    if view:
      print(D)
      print(I)
    if len(D)+len(I)>0:
      repairs += 1
      W, graph = irregularGraphRepairing(V,W,D,I,cr, view)
    
    graph.save("{0}_instance_graphs/gviz_{1}.gv".format(name, num))

    saveGFile(V, W, "{0}_instance_graphs/IG_{1}.g".format(name, num), D, I, time.time()-trace_start_time, sort_labels)
    saveGfinal(V, W, "{0}_instance_graphs.g".format(name), sort_labels)
        
  elapsed = time.time() - start_time_total
  with open("{0}_instance_graphs/{1}.txt".format(name, name), 'w') as f:
    f.write("Execution Time: {0:.3f} s\n".format(elapsed))
    f.write("Number of traces: {0}\n".format(count))
    f.write("Number of repairs: {0}\n".format(repairs))

BIG(net_file5, log_file5)


In [ ]:
 !zip -r /content/BPI2017Denied_instance_graphs.zip /content/BPI2017Denied_instance_graphs

In [ ]:
 !zip -r /content/testBank2000NoRandomNoise.zip /content/testBank2000NoRandomNoise_instance_graphs

In [ ]:
!zip -r /content/bpi2012decompositionExpr.zip /content/bpi2012decompositionExpr_instance_graphs

In [ ]:
!zip -r /content/testBank2000SCCUpdated.zip /content/testBank2000SCCUpdatedCopia_instance_graphs